In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("../")

%matplotlib notebook
import matplotlib.pylab as plt
# pylab.rcParams['figure.figsize'] = (10, 5)

In [2]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
import scipy.optimize as opt

In [3]:
N_NEIGHBORS = 5

# get data

In [4]:
def fkt(x, y):
    return np.sin(-x*y)

In [5]:
def generate_points(n_pts, fkt):
    x = np.random.rand(n_pts)*2-1
    y = np.random.rand(n_pts)*2-1
    z = fkt(x, y)
    return x, y, z

In [6]:
x, y, z = generate_points(2000, fkt)

In [76]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

def visualize_manifold(pts, a, b, path = None, angle=None, vis_path=None):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    if angle is not None:
        ax.view_init(30, angle)

    ax.scatter(x, y, z, alpha=0.2)
    ax.scatter(a[0], a[1], a[2], alpha=1, c='r', s=50)
    ax.scatter(b[0], b[1], b[2], alpha=1, c='r', s=50)

    if path is not None:
        for p in path:
            ax.scatter(p[0], p[1], p[2], alpha=0.5, c='r', s=25)
    
    ax.axis('off')
    
    if vis_path is not None:
        plt.savefig(vis_path)
        print "saved {}".format(vis_path)
        plt.cla()
    else:
        plt.show()

In [30]:
def dist(a, b):
    return np.sqrt(np.sum((a - b)**2))

def proceed(a, b, pts, delta, n_neighbors = 5):
    
    dists = pairwise_distances(np.array([a]), pts)[0]
    idxs = np.argsort(dists)[:n_neighbors]
    
    # reached a
    if np.all(a == b):
        print "finished!"
        return None
    
    # b in nearest neighbors
    if b in pts[idxs]:
        return b
    
    vectors = a - pts[idxs]
    vectors = vectors / np.linalg.norm(vectors, axis=1).reshape(-1, 1)
    vectors = np.nan_to_num(vectors)
    direktion = b - a
    direktion = direktion / np.linalg.norm(direktion)
    
    def objective(w, a, b):
        a_new = a + w.dot(vectors) * delta
        w = w / np.linalg.norm(w)
        return dist(a_new, b)

    lc = opt.minimize(objective, np.ones(n_neighbors), args=(vectors, direktion), 
                      bounds=[[0,1]] * n_neighbors)
        
    w = lc['x']
    
    # random move to overcome local min
    if w.sum() == 0:
        w = -np.random.rand(len(w))
    
    w = w / np.linalg.norm(w)
    
    a_new = a + w.dot(vectors) * delta
    
    
    return a_new

In [84]:
def get_path(beg, end, delta, n_neighbors = 5, vis_path=None, max_len_path=1000):
    
    angle = 0
    
    a = beg.copy()
    b = end.copy()
    
    path = []
    while a is not None:
        path.append(a)
        a = proceed(a, b, pts, delta, n_neighbors = 5)
        
        if vis_path is not None:
            fname = os.path.join(vis_path, str(len(path)) + ".png")
            visualize_manifold(pts, a, b, path=path, vis_path=fname, angle=angle)
            angle += 3
    
        if len(path) > max_len_path:
            print "path did not converge"
            break
    
    print "path contains {} steps".format(len(path))
    
    return path

In [92]:
pts = np.array([x, y, z]).transpose()
a = pts[22]
b = pts[45]
delta = 0.01

In [93]:
import matplotlib
matplotlib.use('Agg')

In [94]:
p = "/export/home/oblum/projects/ld_gan/visualizations/move_in_z_space/1/"
path = get_path(a, b, delta, vis_path=None)

finished!
path contains 268 steps


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:19: RuntimeWarning: invalid value encountered in divide
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:26: RuntimeWarning: invalid value encountered in divide
